<a href="https://colab.research.google.com/github/psotoc/DataKnow1/blob/main/dataKnow_case1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prueba Tecnica DataKnow - Ejercicio 1

Equipo 1: 0.2 x + 0.8 Y

Equipo 2: (x y z) / 3

In [ ]:
import pandas as pd
from pathlib import Path

Path("data/processed").mkdir(parents=True, exist_ok=True)


paths = {
    "X": "data/processed/X_normalized.csv",
    "Y": "data/processed/Y_normalized.csv",
    "Z": "data/processed/Z_normalized.csv",
}

x = pd.read_csv("data/processed/X_normalized.csv", parse_dates=["Date"])
y = pd.read_csv("data/processed/Y_normalized.csv", parse_dates=["Date"])
z = pd.read_csv("data/processed/Z_normalized.csv", parse_dates=["Date"])


print("X:", x["Date"].min().date(), "→", x["Date"].max().date(), "| Filas:", len(x))
print("Y:", y["Date"].min().date(), "→", y["Date"].max().date(), "| Filas:", len(y))
print("Z:", z["Date"].min().date(), "→", z["Date"].max().date(), "| Filas:", len(z))


#display(x.head(3))
#display(y.head(3))
#display(z.head(3))


X: 1988-06-27 → 2024-04-04 | Filas: 9145
Y: 2006-07-11 → 2023-09-12 | Filas: 4486
Z: 2010-01-01 → 2023-08-31 | Filas: 3566


In [ ]:
x_m = x.set_index("Date").sort_index()
y_m = y.set_index("Date").sort_index()
z_m = z.set_index("Date").sort_index()


X_ms = x_m["Price"].resample("MS").mean()
Y_ms = y_m["Price"].resample("MS").mean()
Z_ms = z_m["Price"].resample("MS").mean()

monthly = pd.concat(
    [X_ms.rename("X"), Y_ms.rename("Y"), Z_ms.rename("Z")],
    axis=1
)

monthly = monthly.ffill().bfill()

print(monthly.isna().sum())
print(monthly.index.min(), "→", monthly.index.max())

monthly.to_csv("data/processed/monthly.csv", index=True, date_format="%Y-%m-%d")

print("✔ stored in data/processed/monthly.csv")

check = pd.read_csv("data/processed/monthly.csv", parse_dates=["Date"])
print(check.head(3))
print(check.isna().sum())





X    0
Y    0
Z    0
dtype: int64
1988-06-01 00:00:00 → 2024-04-01 00:00:00
✔ stored in data/processed/monthly.csv
        Date          X      Y            Z
0 1988-06-01  15.172500  541.0  2233.892857
1 1988-07-01  14.924286  541.0  2233.892857
2 1988-08-01  15.009091  541.0  2233.892857
Date    0
X       0
Y       0
Z       0
dtype: int64


In [ ]:
monthly = pd.read_csv("data/processed/monthly.csv", parse_dates=["Date"]).set_index("Date")

monthly["Equipo1"] = 0.2*monthly["X"] + 0.8*monthly["Y"]
monthly["Equipo2"] = (monthly["X"] + monthly["Y"] + monthly["Z"]) / 3

monthly.to_csv("data/processed/teamCosts.csv", index=True, date_format="%Y-%m-%d")

print("✔ stored in data/processed/teamCosts.csv")
display(monthly[["X", "Y", "Z", "Equipo1", "Equipo2"]].head(3))
display(monthly[["X", "Y", "Z", "Equipo1", "Equipo2"]].tail(3))

✔ stored in data/processed/teamCosts.csv


,X,Y,Z,Equipo1,Equipo2
Date,,,,,
1988-06-01,15.172500,541.0,2233.892857,435.834500,930.021786
1988-07-01,14.924286,541.0,2233.892857,435.784857,929.939048
1988-08-01,15.009091,541.0,2233.892857,435.801818,929.967316


,X,Y,Z,Equipo1,Equipo2
Date,,,,,
2024-02-01,81.71619,547.0,2133.021739,453.943238,920.579310
2024-03-01,84.49000,547.0,2133.021739,454.498000,921.503913
2024-04-01,88.71750,547.0,2133.021739,455.343500,922.913080


In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

monthly = pd.read_csv("data/processed/monthly.csv", parse_dates=["Date"]).set_index("Date")

def fit_forecast(series, periods=36):
    series = series.asfreq("MS")  # asegurar frecuencia mensual
    model = ExponentialSmoothing(
        series,
        trend="add",              # tendencia aditiva
        seasonal=None,            # sin estacionalidad
        initialization_method="estimated"
    ).fit()
    forecast = model.forecast(periods)
    return model, forecast

mX, X_f = fit_forecast(monthly["X"], 36)
mY, Y_f = fit_forecast(monthly["Y"], 36)
mZ, Z_f = fit_forecast(monthly["Z"], 36)

forecast_raw = pd.concat([X_f.rename("X_f"), Y_f.rename("Y_f"), Z_f.rename("Z_f")], axis=1)
forecast_raw.to_csv("data/processed/forecast_36.csv", index=True, date_format="%Y-%m-%d")

print("✔ stored in data/processed/forecast_36.csv")
display(forecast_raw.head(3))
display(forecast_raw.tail(3))

✔ stored in data/processed/forecast_36.csv


,X_f,Y_f,Z_f
2024-05-01,88.888531,547.013894,2132.787762
2024-06-01,89.059562,547.027788,2132.553785
2024-07-01,89.230593,547.041682,2132.319809


,X_f,Y_f,Z_f
2027-02-01,94.532559,547.472395,2125.066526
2027-03-01,94.703591,547.486289,2124.832549
2027-04-01,94.874622,547.500183,2124.598573


In [ ]:
forecast_raw = pd.read_csv("data/processed/forecast_36.csv", index_col=0, parse_dates=[0])

forecast_equipos = pd.DataFrame(index=forecast_raw.index)
forecast_equipos["Equipo1_"] = 0.2*forecast_raw["X_f"] + 0.8*forecast_raw["Y_f"]
forecast_equipos["Equipo2_"] = (forecast_raw["X_f"] + forecast_raw["Y_f"] + forecast_raw["Z_f"]) / 3

forecast_equipos.to_csv("data/processed/forecat_costs.csv", index=True, date_format="%Y-%m-%d")
print("✔ stored in data/processed/forecat_costs.csv")

display(forecast_equipos.head(3))
display(forecast_equipos.tail(3))

✔ stored in data/processed/forecat_costs.csv


,Equipo1_,Equipo2_
2024-05-01,455.388821,922.896729
2024-06-01,455.434143,922.880379
2024-07-01,455.479464,922.864028


,Equipo1_,Equipo2_
2027-02-01,456.884428,922.357160
2027-03-01,456.929750,922.340810
2027-04-01,456.975071,922.324459


In [ ]:
Path("reports").mkdir(parents=True, exist_ok=True)


forecast_equipos = pd.read_csv("data/processed/forecat_costs.csv", index_col=0, parse_dates=[0])


resumen = pd.DataFrame({
    "totalCost": [
        forecast_equipos["Equipo1_"].sum(),
        forecast_equipos["Equipo2_"].sum()
    ],
    "men": [
        forecast_equipos["Equipo1_"].mean(),
        forecast_equipos["Equipo2_"].mean()
    ],
    "max": [
        forecast_equipos["Equipo1_"].max(),
        forecast_equipos["Equipo2_"].max()
    ],
    "min": [
        forecast_equipos["Equipo1_"].min(),
        forecast_equipos["Equipo2_"].min()
    ],
}, index=["Equipo1", "Equipo2"])

resumen.to_csv("reports/resumen.csv")

print("✔ stored in data/processed/resumen.csv")
display(resumen)

✔ stored in data/processed/resumen.csv


,totalCost,men,max,min
Equipo1,16422.550064,456.181946,456.975071,455.388821
Equipo2,33213.981390,922.610594,922.896729,922.324459
